# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 

As of today, the easiest way to experiment with Ebiose is to use the OpenAI API. To do so, all you have to do is to set your OpenAI API key via an .env file or by replacing `"your-open-api-key"` in the following code block:

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

> 💡 Recall that you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so or execute the following:


In [3]:
import sys
sys.path.append("./../")


## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [4]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [5]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [ ]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [7]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantitated, we can start generating agents by running a **forge cycle**. 

We must first define a list of available LLMs. Here, we will use GPT 4o mini only. 

In [8]:

from ebiose.core.model_endpoint import ModelEndpoint, ModelSize, ModelType

available_models = [
    ModelEndpoint(
        endpoint_id="gpt-4o-mini",
        model_name="GPT-4o-mini",
    ),
]

In Ebiose, all calls to LLMs (and, in a next future, for any other type of call that costs) are managed by the `ComputeIntensiveBatchProcessor` static class. This class must be initialized with the list of available models, as follows:

In [9]:
from ebiose.compute_intensive_batch_processor.compute_intensive_batch_processor import (
    ComputeIntensiveBatchProcessor,
)
ComputeIntensiveBatchProcessor.initialize(available_models)

The forge cycle is based on an evolutionary algorithm that requires two types of specialized agents: 
- **architect agents** which are agents that generate other agents from scratch;
- **genetic operator agents** which generate agents from one or several other existing agents.

For now, hand-made architect and crossover agents are provided in the `GraphUtils` class. We can load them as follows:

In [10]:
from ebiose.core.engines.graph_engine.utils import GraphUtils

architect_agent = GraphUtils.get_architect_agent(model_endpoint_id="gpt-4o-mini")
crossover_agent = GraphUtils.get_crossover_agent(model_endpoint_id="gpt-4o-mini")

These agents are required to instantiate an Ebiose ecosystem:

In [11]:
from ebiose.core.ecosystem import Ecosystem

ecosystem = Ecosystem(
    initial_architect_agents=[architect_agent],
    initial_genetic_operator_agents=[crossover_agent],
)

The forge cycle can now be launched by providing the created ecosystem, a budget in dollars (the forge cycle will end once this budget is exhausted) and, optionally, a path in which created agents and fitness will be saved accross generations. Note that we need to use `asyncio.run` to launch the forge cycle.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.02 

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents = asyncio.run(forge.run_new_cycle(ecosystem, cycle_config=cycle_config))


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
100%|██████████| 2/2 [01:43<00:00, 51.56s/it]
Agent initialization cost: 0.01754925
Population initialized with 2 agents
Initialization of 2 agents took 0:01:43.152902
Budget left after initialization: 0.0024507500000000015 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2190.81it/s]
Agent agent-12ea5eb3-0dfe-4097-aaf0-65f1ffa047e8 fitness: 0.22786724140880465, cost: 0.0
Agent agent-df86baf9-439d-4cfb-9282-4b079e19d800 fitness: 0.6101299982864118, cost: 0.0
Evaluation took 0:00:00.004704 for a total cost of 0.0 $


# Agent ID: agent-df86baf9-439d-4cfb-9282-4b079e19d800
## Fitness: 0.6101299982864118
```mermaid 
graph LR
	Start_Node[start_node] --> 1(Problem Understanding)
	1(Problem Understanding) --> 2(Extracting Key Information)
	2(Extracting Key Information) --> 3(Formulating the Equation)
	3(Formulating the Equation) --> 4(Solving the Equation)
	4(Solving the Equation) --> 5(Verification of Solution)
	5(Verification of Solution) --> 6(Feedback on Solution)
	6(Feedback on Solution) -->|solution is incorrect| 8(Final Review)
	6(Feedback on Solution) -->|solution is correct| 9(Documentation)
	8(Final Review) --> 9(Documentation)
	9(Documentation) --> 10(Conclusion)
	10(Conclusion) --> End_Node[end_node]
	6(Feedback on Solution) -->|alternative approaches suggested| 7(Alternative Approaches)
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math word problem-solving network. As an LLM, you will play a specific role in processing a math word problem through a series of interconnected nodes. Your task is to contribute meaningfully to the problem-solving process by executing your designated function and accurately transmitting information to the next node. Use chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms to enhance your responses. The overall goal is to arrive at a correct and clear solution to the problem presented. Please ensure that your outputs are comprehensive, insightful, and relevant to the problem-solving journey.
##### Problem Understanding
Analyze the following math word problem carefully. Identify the main elements, such as the subject, context, and any numerical data mentioned. Your goal is to provide a clear and concise summary of what the problem is asking. Please include any sub-questions or key considerations that will guide the next steps in the problem-solving process.
##### Extracting Key Information
From the summary provided by the previous node, extract all relevant numerical data, variables, and relationships. Clearly list the key pieces of information that will be necessary for formulating the mathematical equation. Additionally, highlight any assumptions made while interpreting the problem to ensure clarity for the next node.
##### Formulating the Equation
Based on the key information extracted from the previous node, construct the mathematical equation that represents the problem. Make sure to articulate the reasoning behind your equation formulation, detailing how each piece of information contributes to the overall equation. Ensure that your explanation is clear and can be understood by the next node.
##### Solving the Equation
Using the mathematical equation formulated by the previous node, perform the necessary calculations to derive the solution. Clearly show each step taken during the calculation process. If applicable, explain any mathematical principles or rules used to arrive at the final answer. Provide this solution to the next node in a clear format.
##### Verification of Solution
Evaluate the solution derived from the previous node. Substitute the solution back into the original problem to verify its correctness. Clearly articulate whether the solution fits the context of the problem and meets the requirements specified. If discrepancies are found, outline the specific areas that need further investigation.
##### Feedback on Solution
Based on the verification results, provide constructive feedback regarding the solution. If the solution is correct, commend the previous node and suggest how to present it effectively. If the solution is incorrect or requires improvement, identify specific aspects that need to be re-evaluated and propose actionable steps for refining the solution.
##### Alternative Approaches
Consider the problem-solving process and the solution derived. Suggest alternative methods or approaches that could be applied to solve the math word problem. For each alternative, explain its potential benefits and how it might yield a different perspective on the problem. Ensure that your suggestions are constructive and relevant.
##### Final Review
Conduct a comprehensive review of the entire problem-solving process, from understanding the problem to deriving the solution. Evaluate the clarity, accuracy, and logical flow of the information presented. Highlight any strengths and weaknesses observed, and ensure that the explanation is coherent and understandable for someone unfamiliar with the problem.
##### Documentation
Compile all information from the previous nodes into a structured format that outlines the entire problem-solving journey. Ensure that each section is clearly labeled, and provide a concise summary of the problem, key information, equation formulation, solution, verification, feedback, and alternatives. Your documentation should be clear, informative, and ready for presentation.
##### Conclusion
Based on the documentation created by the previous node, draw a final conclusion regarding the math word problem. Summarize the entire process and the ultimate solution, emphasizing any key takeaways or insights gained from the problem-solving journey. Ensure that your conclusion is impactful and reinforces the significance of the solution reached.

# Agent ID: agent-12ea5eb3-0dfe-4097-aaf0-65f1ffa047e8
## Fitness: 0.22786724140880465
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding)
	Llm_Node_1(Problem Understanding) --> Llm_Node_2(Equation Formulation)
	Llm_Node_2(Equation Formulation) --> Llm_Node_3(Equation Solving)
	Llm_Node_3(Equation Solving) --> Llm_Node_4(Solution Verification)
	Llm_Node_4(Solution Verification) --> Llm_Node_5(Solution Explanation)
	Llm_Node_5(Solution Explanation) -->|solution verified| Llm_Node_6(Feedback and Reflection)
	Llm_Node_5(Solution Explanation) -->|solution not verified| Llm_Node_3(Equation Solving)
	Llm_Node_6(Feedback and Reflection) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a series of well-defined stages. Each node represents a distinct step in the problem-solving process. Your task is to utilize your capabilities to contribute to the overall solution effectively. The nodes include: 1. Understanding the problem, 2. Formulating equations, 3. Solving the equations, 4. Verifying the solution, 5. Explaining the solution, 6. Reflecting on the process. Communicate clearly, share necessary information with the next node, and leverage chain-of-thought reasoning and self-reflection where applicable.
##### Problem Understanding
Read the following math word problem carefully. Identify and extract all key information including the main question, important variables, and any relationships or operations implied in the text. Provide a clear and concise summary of your understanding, including a list of identified variables and their meanings. This summary will be passed to the next node to formulate equations based on your findings.
##### Equation Formulation
Using the information summarized from the previous node, formulate one or more mathematical equations that accurately represent the relationships and operations described in the math word problem. Ensure that each equation corresponds to the variables identified and reflects the problem's requirements. Provide a detailed explanation of how you derived each equation, including any assumptions or interpretations made during the process. This output will be sent to the next node for solving.
##### Equation Solving
Based on the equations provided from the previous node, solve the equations step-by-step. Clearly show your calculations and reasoning for each step taken. If there are multiple possible solutions, present them all and indicate which is most relevant to the original problem. Additionally, note any assumptions or conditions that influenced your solving process, as this information will be critical for the verification stage.
##### Solution Verification
Review the solution derived from the previous node. Verify its correctness by checking the calculations and considering the context of the original math word problem. If the solution appears correct, provide a justification for why it meets the requirements of the problem. If discrepancies arise, identify the errors in the solution or the equations leading to it, and suggest corrective actions or revisions that should be made. Your findings will be used to inform the next node.
##### Solution Explanation
Based on the verified solution and justification provided from the previous node, create a clear and comprehensive explanation of the solution process. Break down the reasoning behind each step, ensuring that anyone reading this explanation can understand how the solution was reached. Include examples or analogies if they help clarify complex concepts. This explanation will aid in solidifying understanding for the final reflective stage.
##### Feedback and Reflection
Reflect on the entire problem-solving process from the beginning to the final explanation. Assess the effectiveness of each stage and identify any areas for improvement. Consider alternative methods that could have been applied at different stages and how they might have led to a better solution or understanding. Summarize your insights and provide constructive feedback for future problem-solving scenarios. This reflection will conclude the process and will be shared with the EndNode.



Saving current state to ../data/2025-03-02_21-08-59/generation=0
Starting crossover and mutation...
Initializing structured output agent for model with fields {'edges': FieldInfo(annotation=list[Edge], required=False, default_factory=list, description='list of edges in the graph'), 'nodes': FieldInfo(annotation=list[Union[StartNode, EndNode, LLMNode, PydanticValidatorNode, RegexRoutingNode]], required=False, default_factory=list, description='list of nodes in the graph'), 'description': FieldInfo(annotation=str, required=False, default=''), 'shared_context_prompt': FieldInfo(annotation=str, required=True, metadata=[MinLen(min_length=1)])} (2)
BudgetExceededError: Master budget limit exceeded. Limit: 0.02, New total: 0.020614499999999997. Finishing process.
Error when calling gpt-4o-mini: Master budget limit exceeded. Limit: 0.02, New total: 0.020614499999999997
BudgetExceededError: Master budget limit exceeded. Limit: 0.02, New total: 0.021962699999999998. Finishing process.
Error when